In [ ]:
import sys
sys.version

In [ ]:
import numpy as np
import skimage.measure
import matplotlib.pyplot as plt

In [ ]:
import pydicom
pydicom.__version__

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rai
rai.__version__

In [ ]:
import raicontours

from raicontours import TG263

raicontours.__version__

In [ ]:
cfg = raicontours.get_config()

In [ ]:
cfg

In [ ]:
rai_starting_model, rai_dependent_model = rai.load_model(cfg=cfg)

In [ ]:
image_paths, structure_path = rai.download_deepmind_example()
structure_path

In [ ]:
x_grid, y_grid, image_stack, image_uids = rai.paths_to_image_stack_hfs(
    cfg=cfg, paths=image_paths
)

In [ ]:
image_stack.shape

In [ ]:
reduced_image_stack = skimage.measure.block_reduce(image_stack, block_size=cfg["reduce_block_sizes"][0], func=np.mean)
reduced_image_stack.shape

In [ ]:
310 / 4

In [ ]:
z = [0, 30, 60, 90]
y = [30, 60, 90, 120]
x = [30, 60, 90]

predicted_masks = rai.inference_over_jittered_grid(
    cfg=cfg, model=rai_starting_model, grid=(z, y, x), image_stack=reduced_image_stack,
)

In [ ]:
reduced_x_grid = skimage.measure.block_reduce(x_grid, block_size=cfg["reduce_block_sizes"][0][2], func=np.mean)
reduced_y_grid = skimage.measure.block_reduce(y_grid, block_size=cfg["reduce_block_sizes"][0][1], func=np.mean)

reduced_predicted_contours_by_structure = rai.masks_to_contours_by_structure(
    cfg=cfg, x_grid=reduced_x_grid, y_grid=reduced_y_grid, masks=predicted_masks
)
rai.plot_contours_by_structure(
    reduced_x_grid, reduced_y_grid, reduced_image_stack, reduced_predicted_contours_by_structure
)

In [ ]:
# predicted_masks = rai.inference_over_jittered_grid(
#     cfg=cfg, model=rai_dependent_model, grid=(z, y, x), image_stack=reduced_image_stack, masks_stack=predicted_masks
# )

In [ ]:
cfg["reduce_block_sizes"]

In [ ]:
upscaled = predicted_masks

for i in range(3):
    upscaled = np.repeat(upscaled, repeats=2, axis=i)

if upscaled.shape[0] != image_stack.shape[0]:
    upscaled = upscaled.astype(np.float32)
    
    upscaled = np.mean([upscaled[1:, ...], upscaled[0:-1, ...]], axis=0)
    assert upscaled.shape[0] == image_stack.shape[0]
    
    upscaled = np.round(upscaled).astype(np.uint8)
    
upscaled.shape

In [ ]:
where_mask = np.where(upscaled > 127.5)
np.min(where_mask, axis=1)

In [ ]:
np.max(where_mask, axis=1)

In [ ]:
reduced_image_stack = skimage.measure.block_reduce(image_stack, block_size=cfg["reduce_block_sizes"][1], func=np.mean)
reduced_image_stack.shape

In [ ]:
z = [0, 30, 60, 90, 120, 150, 180]
y = [60, 90, 120, 150, 180, 210, 240, 270]
x = [70, 100, 130, 160, 190, 220, 250]

predicted_masks = rai.inference_over_jittered_grid(
    cfg=cfg, model=rai_dependent_model, grid=(z, y, x), image_stack=reduced_image_stack, masks_stack=upscaled, max_batch_size=10
)

In [ ]:
reduced_x_grid = skimage.measure.block_reduce(x_grid, block_size=cfg["reduce_block_sizes"][1][2], func=np.mean)
reduced_y_grid = skimage.measure.block_reduce(y_grid, block_size=cfg["reduce_block_sizes"][1][1], func=np.mean)

reduced_predicted_contours_by_structure = rai.masks_to_contours_by_structure(
    cfg=cfg, x_grid=reduced_x_grid, y_grid=reduced_y_grid, masks=predicted_masks
)
rai.plot_contours_by_structure(
    reduced_x_grid, reduced_y_grid, reduced_image_stack, reduced_predicted_contours_by_structure
)

In [ ]:
predicted_masks.shape

In [ ]:
np.max(predicted_masks)

In [ ]:
upscaled = predicted_masks

for i in range(1,3):
    upscaled = np.repeat(upscaled, repeats=2, axis=i)

if upscaled.shape[0] != image_stack.shape[0]:
    upscaled = upscaled.astype(np.float32)
    
    upscaled = np.mean([upscaled[1:, ...], upscaled[0:-1, ...]], axis=0)
    assert upscaled.shape[0] == image_stack.shape[0]
    
    upscaled = np.round(upscaled).astype(np.uint8)
    
upscaled.shape

In [ ]:
structure_ds = pydicom.read_file(structure_path)
[item.ROIName for item in structure_ds.StructureSetROISequence]

In [ ]:
align_map = {
    "Brain": [TG263.Brain],
    "Brainstem": [TG263.Brainstem],
    "Cochlea-Lt": [TG263.Cochlea_L],
    "Cochlea-Rt": [TG263.Cochlea_R],
    "Lacrimal-Lt": [TG263.Glnd_Lacrimal_L],
    "Lacrimal-Rt": [TG263.Glnd_Lacrimal_R],
    "Lens-Lt": [TG263.Lens_L],
    "Lens-Rt": [TG263.Lens_R],
    "Lung-Lt": [TG263.Lung_L],
    "Lung-Rt": [TG263.Lung_R],
    "Mandible": [TG263.Bone_Mandible],
    "Optic-Nerve-Lt": [TG263.OpticNrv_L],
    "Optic-Nerve-Rt": [TG263.OpticNrv_R],
    "Orbit-Lt": [TG263.Eye_L],
    "Orbit-Rt": [TG263.Eye_R],
    "Parotid-Lt": [TG263.Parotid_L],
    "Parotid-Rt": [TG263.Parotid_R],
    "Spinal-Cord": [TG263.SpinalCord],
    "Submandibular-Lt": [TG263.Glnd_Submand_L],
    "Submandibular-Rt": [TG263.Glnd_Submand_R],
}
structure_names = list(align_map.keys())

dicom_contours_by_structure = rai.dicom_to_contours_by_structure(
    ds=structure_ds, image_uids=image_uids, structure_names=structure_names
)



In [ ]:
z = [0, 30, 60, 90, 120]
y = [130, 160, 190, 220, 250, 280, 310, 340]
x = [180, 210, 240, 270, 300, 330]

predicted_masks = upscaled

looped_dice = []
for i in range(5):
    predicted_masks = rai.inference_over_jittered_grid(
        cfg=cfg, model=rai_dependent_model, grid=(z, y, x), image_stack=image_stack, masks_stack=predicted_masks
    )
    
    
    predicted_contours_by_structure = rai.masks_to_contours_by_structure(
        cfg=cfg, x_grid=x_grid, y_grid=y_grid, masks=predicted_masks
    )
    
    aligned_predicted_contours_by_structure = rai.merge_contours_by_structure(
        predicted_contours_by_structure, align_map
    )
    
    dice = {}
    for name in align_map:
        dice[name] = rai.dice_from_contours_by_slice(
            dicom_contours_by_structure[name],
            aligned_predicted_contours_by_structure[name],
        )
        
    looped_dice.append(dice)

In [ ]:
for name in structure_names:
    dice = []
    for item in looped_dice:
        dice.append(item[name])
        
    plt.plot(dice, label=name)
    
plt.legend()

In [ ]:
# predicted_contours_by_structure

In [ ]:
rai.plot_contours_by_structure(
    x_grid, y_grid, image_stack, predicted_contours_by_structure, align_map
)

In [ ]:
renamed_dicom_contours_by_structure = {
    f"DICOM {key}": item for key, item in dicom_contours_by_structure.items()
}

In [ ]:
combined_contours_by_structure = {
    **predicted_contours_by_structure,
    **renamed_dicom_contours_by_structure,
}

rai.plot_contours_by_structure(
    x_grid, y_grid, image_stack, combined_contours_by_structure, align_map
)